In [1]:
import meep as mp
import matplotlib.pyplot as plt
import nlopt
import numpy as np
from meep_material import lorentzfit

In [2]:
resolution = 100
nfreq = 100
fcen = 1.75
df = 3.3

In [3]:
def Material_fit(Material_data_csv=".", 
                   eps_inf=1,
                   fit_wl_min=0.38, fit_wl_max=0.72,
                   num_lorentzians=2, iteration=30
                       ):    
        # 1. 데이터 로드 및 전처리,
        data = np.genfromtxt(Material_data_csv, delimiter=",")
        wl = data[:, 0] * 1e6  # 파장 (µm),
        n_data = data[:, 1] + 1j * data[:, 2]
        eps_all = np.square(n_data) - eps_inf
    
        # 피팅 범위로 데이터 선택,
        mask = (wl >= fit_wl_min) & (wl <= fit_wl_max)
        wl_fit, eps_fit = wl[mask], eps_all[mask]
        freqs_fit = 1 / wl_fit
    
        # 2. Lorentzian 피팅 최적화,
        ps = np.zeros((iteration, 3 * num_lorentzians))
        errors = np.zeros(iteration)
        for m in range(iteration):
            p_rand = [10 ** np.random.random() for _ in range(3 * num_lorentzians)]
            ps[m, :], errors[m] = lorentzfit(p_rand, freqs_fit, eps_fit, nlopt.LD_MMA, 1e-25, 50000)
            print(f"Iteration {m:3d}, error: {errors[m]:.6f}")
        best = np.argmin(errors)
        print(f"Optimal error: {errors[best]:.6f}")
    
        # 3. 최적 파라미터로 Susceptibility 생성 및 Meep 모델 구성,
        suscept = []
        for i in range(num_lorentzians):
            freq_param = ps[best][3*i + 1]
            gamma = ps[best][3*i + 2]
            if freq_param == 0:
                sigma = ps[best][3*i + 0]
                suscept.append(mp.DrudeSusceptibility(frequency=1.0, gamma=gamma, sigma=sigma))
            else:
                sigma = ps[best][3*i + 0] / freq_param**2
                suscept.append(mp.LorentzianSusceptibility(frequency=freq_param, gamma=gamma, sigma=sigma))
    
        return eps_inf, suscept
    
sio2_data_path = "/home/m2ni/EIDL/Tool/Meep/LGD/Material CSV/Al_Palik/Data/CSV file/SiO2/CSV/SiO2_palik_data.csv"
al_data_path   = "/home/m2ni/EIDL/Tool/Meep/LGD/Material CSV/Al_Palik/Data/CSV file/Al_Palik/CSV/Al_palik_data.csv"
andp_data_path = "/home/m2ni/EIDL/Tool/Meep/LGD/Material CSV/Al_Palik/Data/CSV file/aNDP/CSV/aNPD_PFA_data.csv"

eps_inf_sio2, suscept_sio2 = Material_fit(Material_data_csv=sio2_data_path)
eps_inf_al, suscept_al = Material_fit(Material_data_csv=al_data_path)
eps_inf_andp, suscept_andp = Material_fit(Material_data_csv=andp_data_path)

Iteration   0, error: 0.000012
Iteration   1, error: 0.000071
Iteration   2, error: 0.000165
Iteration   3, error: 0.000056
Iteration   4, error: 0.000019
Iteration   5, error: 0.000025
Iteration   6, error: 0.000008
Iteration   7, error: 0.000011
Iteration   8, error: 0.000013
Iteration   9, error: 0.000027
Iteration  10, error: 0.000013
Iteration  11, error: 0.000009
Iteration  12, error: 0.000008
Iteration  13, error: 0.000404
Iteration  14, error: 0.000007
Iteration  15, error: 0.000008
Iteration  16, error: 0.000013
Iteration  17, error: 0.000017
Iteration  18, error: 0.000380
Iteration  19, error: 0.000019
Iteration  20, error: 0.000042
Iteration  21, error: 0.000033
Iteration  22, error: 0.000085
Iteration  23, error: 0.000662
Iteration  24, error: 0.000010
Iteration  25, error: 0.000012
Iteration  26, error: 0.000007
Iteration  27, error: 0.000035
Iteration  28, error: 0.000051
Iteration  29, error: 0.000026
Optimal error: 0.000007
Iteration   0, error: 64.226780
Iteration   1,

In [ ]:
### ----- Refractive index ----- ###,
Glass = mp.Medium(epsilon=eps_inf_sio2, E_susceptibilities = suscept_sio2) # SiO2 ,
aNDP = mp.Medium(epsilon=eps_inf_andp, E_susceptibilities = suscept_andp) # aNDP ,
Cathode = mp.Medium(epsilon = eps_inf_al, E_susceptibilities = suscept_al)

# Cathode = mp.perfect_electric_conductor,

CoverLayer = mp.Medium(index=1.9) # SiN ,
ITO_m = mp.Medium(index=1.8) 
Alq3 = mp.Medium(index=1.68) 
Air = mp.Medium(index=1.0) # 공기,

Glass_layer = 1.37
Cover_layer = 0.07
ITO_layer = 0.15
aNDP_layer = 0.05
Alq3_layer = 0.06
Cathode_layer = 0.3

Absorber_layer = 0.5

boundary_layers = [
    mp.PML(thickness=Absorber_layer, direction=mp.X, side=mp.Low),  # 좌측 PML,
    mp.PML(thickness=Absorber_layer, direction=mp.X, side=mp.High), # 우측 PML,
    mp.PML(thickness=Absorber_layer, direction=mp.Y, side=mp.High), # 상단 PML,
]